# Speech Understanding
## Robust Automatic Speech Recognition in Noisy Environment with Lip-Reading Assistance
### Akansha Gautam    (M23CSA506)
### Anchit Mulye      (M23CSA507)

# Import Libraries

In [2]:
import numpy as np
import pandas as pd
from pytube import YouTube
from datasets import load_dataset
from youtube_transcript_api import YouTubeTranscriptApi
from urllib.parse import urlparse, parse_qs
import subprocess
import whisper
import os
import torch

# Set Device

In [3]:
if torch.backends.mps.is_available():
    device = 'mps'
elif torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

print(f"Using device: {device}")

Using device: mps


# Load AVSpeech Dataset

In [4]:
avspeech_train_path = '/Users/akanshagautam/Documents/MTech/Speech Understanding/Project/dataset/avspeech/avspeech_train.csv'
avspeech_test_path = '/Users/akanshagautam/Documents/MTech/Speech Understanding/Project/dataset/avspeech/avspeech_test.csv'
names = ["youtube_id", "start_segment", "end_segment", "x_coordinate", "y_coordinate"]

avspeech_train_df = pd.read_csv(avspeech_train_path, names=names)
avspeech_test_df = pd.read_csv(avspeech_test_path, names=names)

print(f"AVSpeech Train Dataset Shape: {avspeech_train_df.shape}")
print(f"AVSpeech Test Dataset Shape: {avspeech_test_df.shape}")

avspeech_train_df.head(2)

AVSpeech Train Dataset Shape: (2621845, 5)
AVSpeech Test Dataset Shape: (183273, 5)


,youtube_id,start_segment,end_segment,x_coordinate,y_coordinate
0,CJoOwXcjhds,233.266,239.367000,0.780469,0.670833
1,AvWWVOgaMlk,90.000,93.566667,0.586719,0.311111


# Extract Youtube data

In [5]:
TRAIN_DIR = "/Users/akanshagautam/Documents/MTech/Speech Understanding/Project/dataset/avspeech/train"
MAX_VIDEOS = 100

In [7]:
def extract_youtube_id(url):
    """Extracts video ID from a YouTube URL."""
    query = urlparse(url).query
    return parse_qs(query).get('v', ['video'])[0]

def build_youtube_url(youtube_id):
    return f"https://www.youtube.com/watch?v={youtube_id}"

def download_youtube_media(base_dir, url):
    video_id = extract_youtube_id(url)
    output_dir = os.path.join(base_dir, video_id)

    # Create directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Download video + audio + captions into the folder
    video_command = (
        f'yt-dlp '
        f'-f "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best" '
        f'--merge-output-format mp4 '
        f'--output "{output_dir}/{video_id}.%(ext)s" '
        f'--write-auto-sub '
        f'--sub-lang en '
        f'--convert-subs srt '
        f'{url}'
    )

    # Download audio-only as .wav into the same folder
    audio_command = (
        f'yt-dlp '
        f'--extract-audio '
        f'--audio-format wav '
        f'--output "{output_dir}/{video_id}_audio.%(ext)s" '
        f'{url}'
    )

    # Run the commands
    subprocess.run(video_command, shell=True)
    subprocess.run(audio_command, shell=True)

In [9]:
def build_youtube_url(youtube_id):
    return f"https://www.youtube.com/watch?v={youtube_id}"

def download_youtube_media(youtube_id):
    url = build_youtube_url(youtube_id)
    output_dir = os.path.join(TRAIN_DIR, youtube_id)
    os.makedirs(output_dir, exist_ok=True)

    try:

        # Download audio-only as .wav
        audio_command = (
            f'yt-dlp '
            f'--extract-audio '
            f'--audio-format wav '
            f'--output "{output_dir}/{youtube_id}_audio.%(ext)s" '
            f'{url}'
        )

        # Download video + captions
        video_command = (
            f'yt-dlp '
            f'-f "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best" '
            f'--merge-output-format mp4 '
            f'--output "{output_dir}/{youtube_id}.%(ext)s" '
            f'--write-auto-sub '
            f'--sub-lang en '
            f'--convert-subs srt '
            f'{url}'
        )

        subprocess.run(audio_command, shell=True, check=True)
        audio_file = os.path.join(output_dir, f"{youtube_id}_audio.wav")

        if not is_english_audio(audio_file):
            print(f"Skipping non-English audio: {youtube_id}")
            cleanup(output_dir)
            return False

        subprocess.run(video_command, shell=True, check=True)
        video_file = os.path.join(output_dir, f"{youtube_id}.mp4")
        caption_file = os.path.join(output_dir, f"{youtube_id}.en.srt")

        if not (os.path.exists(video_file) and os.path.exists(audio_file) and os.path.exists(caption_file)):
            print(f"Missing file(s) for {youtube_id}, cleaning up.")
            cleanup(output_dir)
            return False

        print(f"✅ Downloaded: {youtube_id}")
        return True

    except subprocess.CalledProcessError as e:
        print(f"Error downloading {youtube_id}: {e}")
        cleanup(output_dir)
        return False

def cleanup(folder_path):
    """Delete all files in a folder and the folder itself."""
    if os.path.exists(folder_path):
        for f in os.listdir(folder_path):
            os.remove(os.path.join(folder_path, f))
        os.rmdir(folder_path)

def download_dataset(df):
    os.makedirs(TRAIN_DIR, exist_ok=True)

    downloaded_count = len([f for f in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, f))])
    
    for _, row in df.iterrows():
        if downloaded_count >= MAX_VIDEOS:
            print(f"Reached the max limit of {MAX_VIDEOS}.")
            break

        youtube_id = row['youtube_id']
        output_dir = os.path.join(TRAIN_DIR, youtube_id)

        if os.path.exists(output_dir):
            print(f"Skipping existing folder: {youtube_id}")
            continue

        success = download_youtube_media(youtube_id)
        if success:
            downloaded_count += 1

def is_english_audio(audio_path):
    """Returns True if detected audio language is English."""
    try:
        model = whisper.load_model("tiny")  # or "base" for better accuracy
        result = model.transcribe(audio_path, language=None, task="transcribe")
        detected_lang = result.get("language", "")
        return detected_lang == "en"
    except Exception as e:
        print(f"⚠️ Language detection failed: {e}")
        return False

In [ ]:
download_dataset(avspeech_train_df)

/Users/akanshagautam/Documents/MTech/ml_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[youtube] Extracting URL: https://www.youtube.com/watch?v=CJoOwXcjhds
[youtube] CJoOwXcjhds: Downloading webpage
[youtube] CJoOwXcjhds: Downloading tv client config
[youtube] CJoOwXcjhds: Downloading player 179bab65
[youtube] CJoOwXcjhds: Downloading tv player API JSON
[youtube] CJoOwXcjhds: Downloading ios player API JSON


ERROR: [youtube] CJoOwXcjhds: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.
/Users/akanshagautam/Documents/MTech/ml_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Error downloading CJoOwXcjhds: Command 'yt-dlp --extract-audio --audio-format wav --output "/Users/akanshagautam/Documents/MTech/Speech Understanding/Project/dataset/avspeech/train/CJoOwXcjhds/CJoOwXcjhds_audio.%(ext)s" https://www.youtube.com/watch?v=CJoOwXcjhds' returned non-zero exit status 1.
[youtube] Extracting URL: https://www.youtube.com/watch?v=AvWWVOgaMlk
[youtube] AvWWVOgaMlk: Downloading webpage
[youtube] AvWWVOgaMlk: Downloading tv client config
[youtube] AvWWVOgaMlk: Downloading player 179bab65
[youtube] AvWWVOgaMlk: Downloading tv player API JSON
[youtube] AvWWVOgaMlk: Downloading ios player API JSON
[youtube] AvWWVOgaMlk: Downloading m3u8 information
[info] AvWWVOgaMlk: Downloading 1 format(s): 251
[download] Destination: /Users/akanshagautam/Documents/MTech/Speech Understanding/Project/dataset/avspeech/train/AvWWVOgaMlk/AvWWVOgaMlk_audio.webm
[download] 100% of   15.92MiB in 00:00:21 at 763.65KiB/s   
[ExtractAudio] Destination: /Users/akanshagautam/Documents/MTech/Spe

100%|█████████████████████████████████████| 72.1M/72.1M [00:03<00:00, 25.0MiB/s]
/Users/akanshagautam/Documents/MTech/ml_env/lib/python3.9/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


...

[youtube] Extracting URL: https://www.youtube.com/watch?v=5fnzANZN-O4
[youtube] 5fnzANZN-O4: Downloading webpage
[youtube] 5fnzANZN-O4: Downloading tv client config
[youtube] 5fnzANZN-O4: Downloading player 64be519f
[youtube] 5fnzANZN-O4: Downloading tv player API JSON
[youtube] 5fnzANZN-O4: Downloading ios player API JSON
Error downloading 5fnzANZN-O4: Command 'yt-dlp --extract-audio --audio-format wav --output "/Users/akanshagautam/Documents/MTech/Speech Understanding/Project/dataset/avspeech/train/5fnzANZN-O4/5fnzANZN-O4_audio.%(ext)s" https://www.youtube.com/watch?v=5fnzANZN-O4' returned non-zero exit status 1.


ERROR: [youtube] 5fnzANZN-O4: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
/Users/akanshagautam/Documents/MTech/ml_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[youtube] Extracting URL: https://www.youtube.com/watch?v=alHB2f34oRI
[youtube] alHB2f34oRI: Downloading webpage
[youtube] alHB2f34oRI: Downloading tv client config
[youtube] alHB2f34oRI: Downloading player 64be519f
[youtube] alHB2f34oRI: Downloading tv player API JSON
[youtube] alHB2f34oRI: Downloading ios player API JSON
[youtube] alHB2f34oRI: Downloading m3u8 information
[info] alHB2f34oRI: Downloading 1 format(s): 251
[download] Destination: /Users/akanshagautam/Documents/MTech/Speech Understanding/Project/dataset/avspeech/train/alHB2f34oRI/alHB2f34oRI_audio.webm
[download] 100% of    6.75MiB in 00:00:22 at 306.12KiB/s 
[ExtractAudio] Destination: /Users/akanshagautam/Documents/MTech/Speech Understanding/Project/dataset/avspeech/train/alHB2f34oRI/alHB2f34oRI_audio.wav
Deleting original file /Users/akanshagautam/Documents/MTech/Speech Understanding/Project/dataset/avspeech/train/alHB2f34oRI/alHB2f34oRI_audio.webm (pass -k to keep)


/Users/akanshagautam/Documents/MTech/ml_env/lib/python3.9/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/Users/akanshagautam/Documents/MTech/ml_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[youtube] Extracting URL: https://www.youtube.com/watch?v=alHB2f34oRI
[youtube] alHB2f34oRI: Downloading webpage
[youtube] alHB2f34oRI: Downloading tv client config
[youtube] alHB2f34oRI: Downloading player 64be519f
[youtube] alHB2f34oRI: Downloading tv player API JSON
[youtube] alHB2f34oRI: Downloading ios player API JSON
[youtube] alHB2f34oRI: Downloading m3u8 information
[info] alHB2f34oRI: Downloading subtitles: en
[info] alHB2f34oRI: Downloading 1 format(s): 137+140
[info] Writing video subtitles to: /Users/akanshagautam/Documents/MTech/Speech Understanding/Project/dataset/avspeech/train/alHB2f34oRI/alHB2f34oRI.en.vtt
[download] Destination: /Users/akanshagautam/Documents/MTech/Speech Understanding/Project/dataset/avspeech/train/alHB2f34oRI/alHB2f34oRI.en.vtt
[download] 100% of   64.72KiB in 00:00:00 at 477.10KiB/s
[SubtitlesConvertor] Converting subtitles
Deleting original file /Users/akanshagautam/Documents/MTech/Speech Understanding/Project/dataset/avspeech/train/alHB2f34oRI/al